# Import packages 

In [1]:
import pandas as pd
import numpy as np

# Read the .csv

In [180]:
df_job = pd.read_csv('cleaned_datajob.csv', encoding='utf-8-sig')
df_job.head()

,Job_Title,Location,Job_Highlight,JRJD,Company_Name
0,ASSISTANT MANAGER - DATA ANALYTICS,Wan Chai,"Bachelor degree in Computer Science, Informati...","Bachelor degree in Computer Science, Informati...",Sun Hung Kai Properties
1,IT Assistant (Data Management),Mong Kok,Job To perform regular updating of the data an...,Job To perform regular updating of the data an...,EIDEALINK LIMITED
2,"Associate Vice President, Dashboard Reporting ...",Wan Chai,Direct report to Chief Analytics OfficerLead a...,Direct report to Chief Analytics OfficerLead a...,United Asia Finance Limited
3,Data Center Customer Operations Engineer / Fie...,Tsuen Wan Area,Who are we Equinix is one of the fastest growi...,Who are we Equinix is one of the fastest growi...,Equinix Hong Kong Ltd
4,IT Data Centre Operator,Tsuen Wan Area,What you’ll be doing Perform activities for mo...,What you’ll be doing Perform activities for mo...,OCBC Wing Hang Bank Limited


In [181]:
lst_jrjd = df_job.JRJD.tolist()
lst_jobhighligh = df_job.Job_Highlight.tolist()

# NLP Process for job highlight

In [7]:
import spacy
import re

## Entity Ruler For Extracting Experience information 

In [41]:
config = {
   "overwrite_ents": True,
   "validate" : True
}

nlp = spacy.load("en_core_web_sm")
#ruler = nlp.add_pipe('entity_ruler', before="ner", config=config)
ruler = nlp.add_pipe('entity_ruler', after="ner", config=config)
#ruler = nlp.add_pipe('entity_ruler', config=config)

# Min 5-7 year
# 1-2 years working experience
# 10+ years

min_lst = ["min", "min.", "minimium"]



pattern_exp = [
    {"label" : "EXPERIENCE",
     "pattern" : [ {"LOWER": {"IN": min_lst}},
                   {"IS_DIGIT" : True},
                   {"IS_PUNCT" : True, "OP" : "?"},
                   {"IS_DIGIT" : True, "OP" : "?"},
                   {"LEMMA" : "year", "LOWER" : "years","OP" : "+"}]},

    {"label" : "EXPERIENCE",
     "pattern" : [ {"IS_ALPHA" : True, "OP" : "?"},
                   {"IS_DIGIT" : True}, 
                   {"IS_PUNCT" : True, "OP" : "?"},
                   {"IS_DIGIT" : True, "OP" : "?"},
                   {"IS_SPACE" : True, "OP" : "?"},
                   {"IS_SPACE" : True, "OP" : "?"},
                   {"LEMMA" : "year", "LOWER" : "years","OP" : "+"},
                   {"LEMMA" : "of", "LOWER" : "of","OP" : "?"},
                   {"LEMMA" : "work", "LOWER" : "work","OP" : "?"},
                   {"LEMMA" : "experience", "LOWER" : "experience","OP" : "?"}]},

    {"label" : "EXPERIENCE",
     "pattern" : [ {"IS_DIGIT" : True},
                   {"TEXT" : "+", "OP" : "?"},
                   {"IS_SPACE" : True, "OP" : "?"},
                   {"IS_SPACE" : True, "OP" : "?"},
                   {"LEMMA" : "year", "LOWER" : "years","OP" : "+"},
                   {"LEMMA" : "work", "LOWER" : "work","OP" : "?"},
                   {"LEMMA" : "experience", "LOWER" : "experience","OP" : "?"}]},
                
    {"label" : "EXPERIENCE",
     "pattern" : [ {"LOWER" : "at", "OP" : "?"},
                   {"LOWER" : "least", "OP" : "+"},
                   {"IS_DIGIT" : True},
                   {"LEMMA" : "year", "LOWER" : "years","OP" : "+"},
                   {"LEMMA" : "work", "LOWER" : "work","OP" : "?"},
                   {"LEMMA" : "experience", "LOWER" : "experience","OP" : "?"}]}


    ]

ruler.add_patterns(pattern_exp)

In [182]:
text = lst_jrjd[58]
nlp(text)

ent_text = nlp(text)

for ent in ent_text.ents:
    #if ent.label_ == "EXPERIENCE":
    #    print(ent.text, ent.label_)
    print(ent.text, ent.label_)

Analyse FAC
Provide NORP
Data ORG
ETL ORG
Perform Data Analytic ORG
Data Visualization ORG
Data Modelling ORG
Dashboard BuildingJob PERSON
Bachelor ORG
Data Science ORG
Preferably 2 years of EXPERIENCE
Data Analysis ORG
Business Intelligence ORG
SQL / Python ORG
English LANGUAGE
Chinese NORP
theGroup Director ofHuman Resources & Administration ORG
Kerry Logistics PERSON
2614 DATE
the Kerry Group ORG


## Matcher For if the position is open to FG

In [10]:
from spacy.matcher import Matcher

In [18]:
matcher = Matcher(nlp.vocab)

lst_fresh = ['fresh', 'fg', 'undergraduate']
lst_grad = ['grad', 'graduate', 'graduates', 'student','students']

# Welcome FG
pattern_fg = [
    {"LEMMA" : "welcome"},
    {"LOWER" : "to", "OP" : "?"},
    {"LOWER": {"IN": lst_fresh}},
    {"LOWER": {"IN": lst_grad}},
]

matcher.add("ACCEPT_FG", [pattern_fg])

# Open to FG
pattern_fg = [
    {"LEMMA" : "open"},
    { "LEMMA" : "to"},
    {"LOWER": {"IN": lst_fresh}},
    {"LOWER": {"IN": lst_grad}},
]

matcher.add("ACCEPT_FG", [pattern_fg])

# FG is welcome
pattern_fg = [
    {"LOWER" : {"IN" : lst_fresh} },
    {"LOWER" :  {"IN" : lst_grad} },
    {"LOWER" : "is"},
    {"LOWER" : "welcome"}

]

matcher.add("ACCEPT_FG", [pattern_fg])

# FG 
pattern_fg = [
    {"LOWER" : {"IN" : lst_fresh} },
    {"LOWER" :  {"IN" : lst_grad} }
]

matcher.add("ACCEPT_FG", [pattern_fg], greedy="LONGEST")

In [25]:
text = lst_jobhighligh[39]
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])

for match in matches:
    print (doc[match[1]:match[2]])
    print (match)



Welcome to Undergraduate students
(6273632314162797395, 5, 9)


## Capture the education requirement

In [203]:
lst_related = ["related", "relevant", "equivalent", "stream"]
lst_role = ["graduate", "student"]

# degree in xx
# bachelor degree in xxx, xxxx, xxxx
# bachelor degree in xxx, xxxx, xxxx or related
# degree holder in xxx, or related disciplines
#Bachelor Degree in IT or related discipline5
pattern_edu =[
    {"LOWER" : "bachelor", "OP" : "?"},
    {"LOWER" : "degree"},
    {"LOWER" : "in"},
    {"IS_ALPHA" : True, "OP" : "+"},
    {"LOWER" : {"IN" : lst_related}}
]

matcher.add("EDU", [pattern_edu], greedy="LONGEST")



# bachelor’s degree in any discipline
pattern_edu =[
    {"LOWER" : "bachelor", "OP" : "?"},
    {"LOWER" : "degree"},
    {"LOWER" : "in"},
    {"LOWER" : "any"},
    {"LEMMA" : "discipline"}
]

matcher.add("EDU", [pattern_edu], greedy="LONGEST")



# Bachelor or Master Degree in xxx or equivalent
pattern_edu =[
    {"LOWER" : "bachelor"},
    {"LOWER" : "or", "OP" : "?"},
    {"LOWER" : "master"},
    {"LEMMA" : "degree", "OP" :  "?"},
    {"LOWER" : "in", "OP" :  "?"},
    {"IS_ALPHA" : True, "OP" : "+"},
    {"LOWER" : {"IN" : lst_related}}
]

matcher.add("EDU", [pattern_edu], greedy="LONGEST")



# University graduate in xxx or in relevant
pattern_edu =[
    {"LEMMA" : "university"},
    {"LEMMA" : {"IN" : lst_role}},
    {"IS_ALPHA" : True, "OP" : "+"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"LOWER" : {"IN" : lst_related}}
]

matcher.add("EDU", [pattern_edu], greedy="LONGEST")



# year + graduate in xxxx 
pattern_edu =[
    {"IS_DIGIT" : True, "OP" : "?"},
    {"LEMMA" : {"IN" : lst_role}},
    {"IS_ALPHA" : True, "OP" : "+"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"IS_PUNCT" : True, "OP" : "*"},
    {"IS_ALPHA" : True, "OP" : "*"},
    {"LOWER" : {"IN" : lst_related}}
]

matcher.add("EDU", [pattern_edu], greedy="LONGEST")


# related: related, relevant, equivalent

In [195]:
lst_jrjd[42]

"Our global clients, inclusive community, flexible and caring culture provide you with opportunities from day one, set you up for long-term success with flexibility, resources and connections. It’s how we help you create a career that you will love and embrace. At your selected service line, you will learn and adapt to the firm’s approach, practices and methodologies in performing selected procedures and documentation under close supervision of seniors. You will become a member of the engagement team and work alongside partners and senior executives to support the respective assignments of various nature and clients including MNCs, private and listed corporations.2023 graduates in Accounting / MIS / Finance / Business stream with a passion to learn and excelInternship or part-time expereince a definite advantageA positive team player with excellent analytical and inter-personal skillsDemonstrated project management skillset including project planning and time managementCan communicate 

In [198]:
text = lst_jrjd[42]
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])

for match in matches:
    print (doc[match[1]:match[2]])
    print (match)



graduates in Accounting / MIS / Finance / Business stream with a passion to learn and excelInternship or part-time expereince a definite advantageA positive team player with excellent analytical and inter-personal skillsDemonstrated project management skillset including project planning and time managementCan communicate effectively in written and spoken English, Chinese and PutonghuaAble to work efficiently and effectively in a complex team environmentWe are a Caring Employer who offers attractive remuneration packages including medical insurance, a
(1612162546529609966, 115, 195)


In [204]:
text = "graduates in Accounting / MIS / Finance / Business stream with a passion to learn and excelInternship"
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])

for match in matches:
    print (doc[match[1]:match[2]])
    print (match)

graduates in Accounting / MIS / Finance / Business stream with a passion to learn and excelInternship
(1612162546529609966, 0, 17)


In [58]:
lst_jrjd[61:90]

['Our client is one of the largest property developers with heavy involvement in investment, leasing and development. They are currently seeking a high calibre data engineer to join their data tech team to cover multiple sophisticated projects related to smart hotel and smart building.  Support overall data platform architect, optimize data sets, data pipelinesBuild required infrastructure for optimal ETL from multiple data sourcesSupport data transformation, data structures, and metadataBuild analytic tools to fully utilize data pipeline and provide meaningful metrics for operation efficiencyLiaise with system and development teams to implement production environment  University Degree in Data Science, Computer Science, Information Technology, Statistics or relevantProgramming Languages  Scala (Must) and Python (Plus)3-5 years related experience on software development on enterprise grade productExperience on Data Modelling, Handling, and Data APISoftware development experience with S

## Capture required skill

In [ ]:
# SAS, Oracle
# SQL, MSSQL, MS SQL , PostgreSQL, MySQL
# BI tools / Power BI / Tableau
# Microsoft Azure Cloud, Azure, AWS
# Python, R, 

In [52]:
str = "Data Mining\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0"

test = str.replace("\xa0", '')

print(test)

Data Mining
